<a href="https://colab.research.google.com/github/ekaropolus/recommender_and_graph_spp/blob/main/NLP_Neo4j_ExpertIA_Sustainable_Public_Policies_MX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install neo4j
! pip install expertai-nlapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 173 kB 31.8 MB/s 
  Created wheel for neo4j: filename=neo4j-5.1.0-py3-none-any.whl size=248097 sha256=869ea581622d5cbe948bd4d17ed3eb62f335133e90bb4ccc0414ac499f60be56
  Stored in directory: /root/.cache/pip/wheels/64/96/da/c30ec8577a970eadecc436103fbcbf7af1d6398d0ebe32cbab
Successfully built neo4j
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 5.2 MB/s 


In [4]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import pandas as pd
import json
import os
from expertai.nlapi.cloud.client import ExpertAiClient


In [5]:

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def find_public_policy(self):
        with self.driver.session(database="neo4j") as session:
            result = session.read_transaction(self._find_and_return_public_policy)
            return result

    @staticmethod
    def _find_and_return_public_policy(tx):
        query = (
            "MATCH (n) "
            "WHERE n.Country = 'MX' "
            "RETURN n.PublicPolicyId as id, n.Level as group, n.Description as description"
        )
        result = tx.run(query)
        return [{"id":row["id"],"group":row["group"],"description":row["description"].lower()} for row in result]

    def find_public_policy_relations(self):
        with self.driver.session(database="neo4j") as session:
            result = session.read_transaction(self._find_and_return_public_policy_relations)
            return result

    @staticmethod
    def _find_and_return_public_policy_relations(tx):
        query = (
            "MATCH (n)-[r:DIVIDED_IN]->(m) "
            "WHERE n.Country = 'MX' "
            "RETURN n.PublicPolicyId as source, m.PublicPolicyId as target, (n.Level + m.Level) as value"
        )
        result = tx.run(query)
        return [(row["source"],row["target"],row["value"]) for row in result]



In [6]:
# Aura queries use an encrypted connection using the "neo4j+s" URI scheme
uri = "neo4j+s://2c2e2321.databases.neo4j.io"
user = "neo4j"
password = "kPJiOCU2ekZYW7159DH_2-4-638msHWntxaWbRkxGVQ"
app = App(uri, user, password)


In [7]:
os.environ["EAI_USERNAME"] = 'hi@hadox.org'
os.environ["EAI_PASSWORD"] = 'RYD.Lq5Et4nT4T4'
client = ExpertAiClient()
language= 'es'

In [ ]:
li_pp_result = app.find_public_policy()
#li_pp_relations_result = app.find_public_policy_relations()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: read_transaction has been renamed to execute_read
  if sys.path[0] == '':


In [ ]:
print(len(li_pp_result))

2172


In [ ]:
iter = 0
for x in li_pp_result:
  iter = iter + 1
  text = x["description"]
  output = client.full_analysis(body={"document": {"text": text}}, params={'language': language})
  lemmas = []
  for y in output.main_lemmas:
    lemma ={"value":y.value,"score":y.score}
    lemmas.append(lemma)

  lemmas_json = json.dumps(lemmas, indent=2)
  x["lemmas"] = lemmas #lemmas_json
  
  syncons = []
  for z in output.main_syncons:
    syncon ={"lemma":z.lemma,"score":z.score}
    syncons.append(syncon)

  syncons_json = json.dumps(syncons, indent=2)
  x["syncons"] = syncons #syncons_json
  

In [ ]:
for x in li_pp_result:
  for y in x["lemmas"]:
    y["id"] = x["id"]
  for y in x["syncons"]:
    y["id"] = x["id"]


In [ ]:
df_pp = pd.DataFrame.from_dict(li_pp_result)

In [ ]:
df_pp.head()

,id,group,description,lemmas,syncons
0,AG1,1,"i. aguascalientes educado, integrado y equitativo","[{'value': 'educado', 'score': 44.09, 'id': 'A...","[{'lemma': 'cortés', 'score': 43.09, 'id': 'AG..."
1,AG2,1,"ii. aguascalientes derecho, seguro y libre","[{'value': 'ii. aguascalientes', 'score': 99.0...","[{'lemma': 'derecho', 'score': 5.69, 'id': 'AG..."
2,AG3,1,"iii. aguascalientes con gobierno íntegro, aust...","[{'value': 'gobierno', 'score': 46.59, 'id': '...","[{'lemma': 'ministerio', 'score': 51.59, 'id':..."
3,AG4,1,"iv. aguascalientes competitivo, diversificado ...","[{'value': 'diversificado', 'score': 43.79, 'i...","[{'lemma': 'diversificado', 'score': 43.29, 'i..."
4,AG5,1,"v. aguascalientes responsable, sostenible y li...","[{'value': 'v. aguascalientes', 'score': 90.19...","[{'lemma': 'diligente', 'score': 6.5, 'id': 'A..."


In [ ]:
%cd /content/drive/MyDrive/Proyectos 2022/01 Hackahton Expert.ia Neurolitks

/content/drive/MyDrive/Proyectos 2022/01 Hackahton Expert.ia Neurolitks


In [ ]:

df_pp.to_parquet('df_pp_2.parquet.gzip',
              compression='gzip')  

In [ ]:
df_pp.to_csv("df_pp_2.csv",index=False)

In [ ]:
from itertools import chain
df_lemmas = pd.DataFrame(list(chain.from_iterable(df_pp["lemmas"])))

In [ ]:
df_lemmas.head()

,value,score,id
0,educado,44.09,AG1
1,integrado,38.59,AG1
2,equitativo,15.10,AG1
3,ii. aguascalientes,99.00,AG2
4,gobierno,46.59,AG3


In [ ]:
df_syncons = pd.DataFrame(list(chain.from_iterable(df_pp["syncons"])))

In [ ]:
df_syncons.head()

,lemma,score,id
0,cortés,43.09,AG1
1,integrado,38.40,AG1
2,justo,15.69,AG1
3,derecho,5.69,AG2
4,cierto,5.40,AG2


In [ ]:
df_lemmas["value"].value_counts().to_csv("values_lemas.csv")

In [ ]:
df_syncons["lemma"].value_counts().to_csv("lemma_syncons.csv")

In [8]:
  text = "Tiene dos propósitos: El primero, hacia la inclusión social. Es decir, hacia una cultura que promocione, reconozca, garantice o restituya los derechos y deberes de las personas con discapacidad y sus familias. La inclusión social implica acceder, disponer, aportar y transformar las condiciones que determinan la desigualdad. Implica también, reconocer y hacer uso de los recursos conceptuales, técnicos y metodológicos que contribuyen a reconstruir los lazos sociales y culturales para disfrutar de la vida y de la ciudad.                                                       El segundo propósito, hacia la calidad de vida con dignidad. Es decir, hacia la búsqueda del bienestar de las personas con discapacidad –PCD-, sus familias, cuidadoras y cuidadores, mediante la satisfacción de necesidades que permitan conseguir una vida digna y libre desde las perspectivas: humana, social"
  output = client.full_analysis(body={"document": {"text": text}}, params={'language': language})

In [11]:
output.main_lemmas

In [12]:
df = pd.DataFrame([vars(f) for f in output.main_lemmas])

In [13]:
df

,_value,_score,_positions
0,propósito,14.69,[<expertai.nlapi.common.model.position.Positio...
1,inclusión,11.89,[<expertai.nlapi.common.model.position.Positio...
2,discapacidad,8.39,[<expertai.nlapi.common.model.position.Positio...
3,deber,7.69,[<expertai.nlapi.common.model.position.Positio...
4,condiciones,6.90,[<expertai.nlapi.common.model.position.Positio...
5,necesidad,6.69,[<expertai.nlapi.common.model.position.Positio...
6,bienestar,6.30,[<expertai.nlapi.common.model.position.Positio...


In [14]:
fields = ['value', 'score']
df = pd.DataFrame([{fn: getattr(f, fn) for fn in fields} for f in output.main_lemmas])

In [15]:
df

,value,score
0,propósito,14.69
1,inclusión,11.89
2,discapacidad,8.39
3,deber,7.69
4,condiciones,6.90
5,necesidad,6.69
6,bienestar,6.30
